In [43]:
import pandas as pd
import numpy as np
import gzip
import timeit

In [44]:
df_cases = pd.read_pickle('df_cases_200906.gzip')

In [45]:
df_labels = pd.read_pickle('df_label_200906.gzip')

In [46]:
df_cases_copy = df_cases

In [47]:
df_cases_copy_selected = df_cases_copy[['CaseId','FileName', 'IsExecuted', 'OcrText', 'QualityScore']]

In [48]:
df_cases_copy_selected.head()

,CaseId,FileName,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-3...,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0...,False,Soon especially boy thousand traditional.,0.873038
2,003061226383,003061226383_41775424_other documents_978-1-38...,True,Tax south say strategy hard between late.,0.991280
3,003061227777,003061227777_96463321_other documents_978-1-96...,False,Yet arm six design.,0.607833
4,003061227781,003061227781_62201704_other documents_978-1-04...,False,Plan camera actually run push rest.,0.909359


In [49]:
df_cases_copy_selected = df_cases_copy_selected.groupby('CaseId').agg({'FileName':lambda series: list(series), 'IsExecuted':lambda series: list(series), 'IsExecuted':lambda series: list(series), 'OcrText':lambda series: list(series), 'QualityScore':lambda series: list(series)})

In [50]:
df_cases_copy_selected.reset_index(inplace=True)

In [51]:
df_cases_copy_selected.head()

,CaseId,FileName,IsExecuted,OcrText,QualityScore
0,003061189006,[003061189006_69176036_Order form_978-1-62414-...,[True],[As difficult behavior her myself help.],[0.8552299535045276]
1,003061189067,[003061189067_26173467_Order form_978-1-904782...,[True],[Present can phone form.],[0.8490947313000836]
2,003061189156,[003061189156_47966765_Contract Documents_978-...,[True],[Section science difference success wish it wi...,[0.8946672829756795]
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,"[False, True]","[Down maintain Republican., Accept sell leader...","[0.6162297231459162, 0.90518190227276]"
4,003061189242,[003061189242_91658807_other documents_978-0-7...,[True],[Our his understand this listen fish.],[0.7986444559587554]


In [52]:
df_cases_copy_selected['InvalidFileNames'] = np.empty((len(df_cases_copy_selected), 0)).tolist()
df_cases_copy_selected['ValidFileNames'] = np.empty((len(df_cases_copy_selected), 0)).tolist()
df_cases_copy_selected['OcrTextCombined'] = ''

In [53]:
df_cases_copy_selected.head()

,CaseId,FileName,IsExecuted,OcrText,QualityScore,InvalidFileNames,ValidFileNames,OcrTextCombined
0,003061189006,[003061189006_69176036_Order form_978-1-62414-...,[True],[As difficult behavior her myself help.],[0.8552299535045276],[],[],
1,003061189067,[003061189067_26173467_Order form_978-1-904782...,[True],[Present can phone form.],[0.8490947313000836],[],[],
2,003061189156,[003061189156_47966765_Contract Documents_978-...,[True],[Section science difference success wish it wi...,[0.8946672829756795],[],[],
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,"[False, True]","[Down maintain Republican., Accept sell leader...","[0.6162297231459162, 0.90518190227276]",[],[],
4,003061189242,[003061189242_91658807_other documents_978-0-7...,[True],[Our his understand this listen fish.],[0.7986444559587554],[],[],


In [54]:
def valid_ocr_text(x):
    valid_ocrtext= ''

    for j in range(0, len(x['IsExecuted'])):
        if ((x['IsExecuted'][j]==False) or (x['QualityScore'][j] < 0.81)):
            x['InvalidFileNames'].append(x['FileName'][j])
        else:
            x['ValidFileNames'].append(x['FileName'][j])
            valid_ocrtext = valid_ocrtext + x['OcrText'][j]

        x['OcrTextCombined'] = valid_ocrtext

In [55]:
df_cases_copy_selected.apply(lambda x: valid_ocr_text(x), axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
1093    None
1094    None
1095    None
1096    None
1097    None
Length: 1098, dtype: object

In [64]:
df_cases_copy_selected.head()

,CaseId,FileName,IsExecuted,OcrText,QualityScore,InvalidFileNames,ValidFileNames,OcrTextCombined
0,003061189006,[003061189006_69176036_Order form_978-1-62414-...,[True],[As difficult behavior her myself help.],[0.8552299535045276],[],[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.
1,003061189067,[003061189067_26173467_Order form_978-1-904782...,[True],[Present can phone form.],[0.8490947313000836],[],[003061189067_26173467_Order form_978-1-904782...,Present can phone form.
2,003061189156,[003061189156_47966765_Contract Documents_978-...,[True],[Section science difference success wish it wi...,[0.8946672829756795],[],[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,"[False, True]","[Down maintain Republican., Accept sell leader...","[0.6162297231459162, 0.90518190227276]",[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.
4,003061189242,[003061189242_91658807_other documents_978-0-7...,[True],[Our his understand this listen fish.],[0.7986444559587554],[003061189242_91658807_other documents_978-0-7...,[],


In [57]:
df_cases_copy_selected['InvalidFileNames'][1094]

['003061230710_80047544_other documents_978-1-02-279791-8.pdf',
 '003061230710_59256366_Order form_978-1-209-37083-5.pdf']

In [58]:
df_labels_copy = df_labels

In [59]:
merged_inner = pd.merge(left=df_cases_copy_selected, right=df_labels_copy, left_on='CaseId', right_on='CaseId')

In [60]:
merged_inner.drop(columns=['FileName', 'IsExecuted', 'OcrText', 'QualityScore'], inplace=True)

In [61]:
merged_inner.rename(columns={'OcrTextCombined': 'OcrText'}, inplace=True)

In [65]:
merged_inner.head()

,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
0,003061189006,[],[003061189006_69176036_Order form_978-1-62414-...,As difficult behavior her myself help.,True,False
1,003061189067,[],[003061189067_26173467_Order form_978-1-904782...,Present can phone form.,True,False
2,003061189156,[],[003061189156_47966765_Contract Documents_978-...,Section science difference success wish it wide.,False,False
3,003061189229,[003061189229_69115288_Master contract_978-0-1...,[003061189229_21094545_other documents_978-1-8...,Accept sell leader herself if.,False,False
4,003061189242,[003061189242_91658807_other documents_978-0-7...,[],,False,False


In [63]:
merged_inner['InvalidFileNames'][1094]

['003061230710_80047544_other documents_978-1-02-279791-8.pdf',
 '003061230710_59256366_Order form_978-1-209-37083-5.pdf']

In [66]:
merged_inner.to_csv("df_final.gzip", index=False, compression="gzip")